In [2]:
import pandas as pd
from sqlalchemy import create_engine

In [3]:
filepath = "Resources\GunViolenceDataset2022.csv"
cars_df = pd.read_csv(filepath, encoding="UTF-8")
cars_df.head()

,Incident ID,Incident Date,state,City_Or_County,Address,n_killed,n_injured
0,2202397,1/1/2022,Indiana,Indianapolis,3800 block of N Temple Ave,0,1
1,2202390,1/1/2022,Michigan,Detroit,Joy Rd and Prest St,0,1
2,2205160,1/1/2022,North Carolina,Summerfield,Spotswood Rd and Ashview Ct,0,1
3,2211626,1/1/2022,Kansas,Lawrence,1837 N 1500 Rd,0,1
4,2206646,1/1/2022,Oklahoma,Oklahoma City,S McKinley Ave and SW 27th St,1,1
